In [1]:
%load_ext nb_black
# !pip install nb_black
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE
from multiscorer import MultiScorer
from sklearn.model_selection import cross_val_score
from numpy import average
import xgboost as xgb
from sklearn import metrics as m
import itertools

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [3]:
randomseed = 7
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [24]:
data = pd.read_csv("test-temp.csv")
originalFeatures = [
    "FPG",
    "HbA1c",
    "Triglyceride",
    "BMI",
    "r-GTP gamma",
    "Uric acid",
    "Smoking",
    "Drinking",
    "Physical activity",
    "Family history",
    "SEX",
    "AGE",
]

<IPython.core.display.Javascript object>

In [25]:
data = data.sort_values(by="Model")
data

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
68,"['FPG', 'r-GTP gamma']",0.722,0.732,0.722,0.726,0.743,0.014,RF
46,"['FPG', 'HbA1c', 'Triglyceride', 'Physical act...",0.747,0.753,0.747,0.749,0.767,0.015,RF
22,"['FPG', 'HbA1c', 'Triglyceride', 'BMI', 'r-GTP...",0.750,0.758,0.750,0.753,0.750,0.038,RF
45,"['FPG', 'HbA1c', 'Drinking', 'Family history',...",0.750,0.755,0.750,0.752,0.760,0.016,RF
24,"['FPG', 'HbA1c', 'BMI', 'r-GTP gamma', 'Smokin...",0.748,0.755,0.748,0.751,0.745,0.037,RF
...,...,...,...,...,...,...,...,...
48,"['FPG', 'HbA1c', 'Smoking', 'Physical activity...",0.747,0.747,0.747,0.747,0.737,0.015,XGB
47,"['FPG', 'HbA1c', 'Uric acid', 'Physical activi...",0.747,0.747,0.747,0.747,0.736,0.014,XGB
40,"['FPG', 'HbA1c', 'Smoking', 'Physical activity...",0.747,0.748,0.747,0.747,0.736,0.015,XGB
59,"['FPG', 'HbA1c', 'Drinking', 'SEX']",0.743,0.744,0.743,0.743,0.738,0.012,XGB


<IPython.core.display.Javascript object>

In [26]:
for i in range(data.shape[0]):
    temp = list(
        str(data.iloc[i, 0])
        .replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .split(",")
    )
    temp_changed = []
    for j in range(len(temp)):
        temp_changed.append(originalFeatures.index(temp[j].strip()))
    data.iloc[i, 0] = str(temp_changed)
data

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
68,"[0, 4]",0.722,0.732,0.722,0.726,0.743,0.014,RF
46,"[0, 1, 2, 8, 10]",0.747,0.753,0.747,0.749,0.767,0.015,RF
22,"[0, 1, 2, 3, 4, 6, 10, 11]",0.750,0.758,0.750,0.753,0.750,0.038,RF
45,"[0, 1, 7, 9, 10]",0.750,0.755,0.750,0.752,0.760,0.016,RF
24,"[0, 1, 3, 4, 6, 9, 10, 11]",0.748,0.755,0.748,0.751,0.745,0.037,RF
...,...,...,...,...,...,...,...,...
48,"[0, 1, 6, 8, 10]",0.747,0.747,0.747,0.747,0.737,0.015,XGB
47,"[0, 1, 5, 8, 10]",0.747,0.747,0.747,0.747,0.736,0.014,XGB
40,"[0, 1, 6, 8, 9, 10]",0.747,0.748,0.747,0.747,0.736,0.015,XGB
59,"[0, 1, 7, 10]",0.743,0.744,0.743,0.743,0.738,0.012,XGB


<IPython.core.display.Javascript object>

In [27]:
# test_list = (
#     (data.iloc[0, 0]).replace("[", "").replace("]", "").replace("'", "").split(",")
# )
# print(test_list)
# test_list = [int(i) for i in test_list]
# test_list

<IPython.core.display.Javascript object>

In [28]:
Rf_list = (
    data.query("Model=='RF'")
    .sort_values(by="CV_Accuracy", ascending=False)
    .reset_index(drop=True)
)
xgb_list = (
    data.query("Model=='XGB'")
    .sort_values(by="CV_Accuracy", ascending=False)
    .reset_index(drop=True)
)

svm_list = (
    data.query("Model=='SVM'")
    .sort_values(by="CV_Accuracy", ascending=False)
    .reset_index(drop=True)
).iloc[:10, :]

svm_list

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
0,"[0, 1, 6, 7, 8, 9, 10]",0.747,0.744,0.747,0.745,0.729,0.011,SVM
1,"[0, 1, 3, 4, 6, 7, 8, 9, 10, 11]",0.740,0.737,0.740,0.738,0.727,0.031,SVM
2,"[0, 1, 3, 4, 6, 7, 8, 10, 11]",0.740,0.737,0.740,0.738,0.726,0.030,SVM
3,"[0, 1, 3, 4, 5, 6, 7, 9, 10, 11]",0.742,0.739,0.742,0.740,0.726,0.029,SVM
4,"[0, 1, 2, 3, 4, 5, 6, 7, 10, 11]",0.738,0.736,0.738,0.737,0.726,0.029,SVM
5,"[0, 1, 2, 3, 4, 5, 6, 7, 9, 11]",0.737,0.734,0.737,0.735,0.726,0.030,SVM
6,"[0, 1, 3, 4, 5, 6, 7, 8, 11]",0.742,0.739,0.742,0.740,0.725,0.030,SVM
7,"[0, 1, 2, 8, 10]",0.745,0.742,0.745,0.743,0.724,0.010,SVM
8,"[0, 1, 2, 10]",0.742,0.739,0.742,0.740,0.724,0.010,SVM
9,"[0, 1, 7, 9, 10]",0.747,0.744,0.747,0.745,0.724,0.010,SVM


<IPython.core.display.Javascript object>

In [29]:
Rf_list

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
0,"[0, 1, 5, 10]",0.743,0.754,0.743,0.747,0.769,0.017,RF
1,"[0, 1, 2, 8, 10]",0.747,0.753,0.747,0.749,0.767,0.015,RF
2,"[0, 1, 6, 7]",0.743,0.757,0.743,0.748,0.762,0.017,RF
3,"[0, 1, 7, 10]",0.748,0.756,0.748,0.751,0.760,0.018,RF
4,"[0, 1, 7, 9, 10]",0.750,0.755,0.750,0.752,0.760,0.016,RF
5,"[0, 1, 6, 8, 9, 10]",0.748,0.756,0.748,0.751,0.759,0.017,RF
6,"[0, 1]",0.725,0.737,0.725,0.730,0.758,0.022,RF
7,"[0, 1, 2, 3, 4, 5, 6, 9, 10, 11]",0.737,0.743,0.737,0.739,0.753,0.037,RF
8,"[0, 1, 3, 4, 5, 6, 7, 8, 11]",0.742,0.751,0.742,0.745,0.753,0.039,RF
9,"[0, 1, 2, 3, 4, 5, 8, 9, 10, 11]",0.737,0.741,0.737,0.739,0.752,0.038,RF


<IPython.core.display.Javascript object>

 2.Read dataset
 ===

In [9]:
# read the data set
x_original = pd.read_csv("../../../../dataset/XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
    ]
]

print(x_original.shape)

y_original = pd.read_csv("../../../../dataset/TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "CLASS"]]

print(y_original.shape)

data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)



# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia

print(data.shape)

data = data[
    [
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data = data.dropna()
print(data.shape)
data.head()



(185843, 19)
(185843, 2)
(169024, 20)
(56542, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
20,87.0,5.60,340.0,24.6,26.0,4.7,1.0,0.0,2.0,0.0,1.0,59.0,0


<IPython.core.display.Javascript object>

In [10]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1045 17331 38166


<IPython.core.display.Javascript object>

In [11]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [12]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [14]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTE(random_state=randomseed, sampling_strategy="minority")
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 17131, 1: 17131, 0: 17131})
17131 17131 17131
(51393, 12) (51393,)


<IPython.core.display.Javascript object>

3.Train and Generate models 
===

In [30]:
weakmodles = []
estimators = []


<IPython.core.display.Javascript object>

In [31]:
def changelist(strlist):
    test_list = (strlist).replace("[", "").replace("]", "").replace("'", "").split(",")
    test_list = [int(i) for i in test_list]
    return test_list

<IPython.core.display.Javascript object>

In [32]:
rf_12 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100,
    max_depth=12,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)

xgb_model_12 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)

scv_12 = svmgpu(
    C=100,
    cache_size=None,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=None,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

<IPython.core.display.Javascript object>

In [33]:
for i in range(len(Rf_list)):
    weakmodles.append(
        make_pipeline(ColumnSelector(cols=changelist(Rf_list.iloc[i, 0])), rf_12)
    )
    estimators.append(
        (
            (
                "rf_12" + str(i),
                (
                    make_pipeline(
                        ColumnSelector(cols=changelist(Rf_list.iloc[i, 0])), rf_12
                    )
                ),
            )
        )
    )

<IPython.core.display.Javascript object>

In [34]:
for i in range(len(xgb_list)):
    weakmodles.append(
        make_pipeline(
            ColumnSelector(cols=changelist(xgb_list.iloc[i, 0])), xgb_model_12
        )
    )
    estimators.append(
        (
            (
                "xgb_model_12" + str(i),
                (
                    make_pipeline(
                        ColumnSelector(cols=changelist(xgb_list.iloc[i, 0])),
                        xgb_model_12,
                    )
                ),
            )
        )
    )

<IPython.core.display.Javascript object>

In [20]:
for i in range(len(svm_list)):
    weakmodles.append(
        make_pipeline(ColumnSelector(cols=changelist(svm_list.iloc[i, 0])), scv_12)
    )
    estimators.append(
        (
            (
                "scv_12" + str(i),
                (
                    make_pipeline(
                        ColumnSelector(cols=changelist(svm_list.iloc[i, 0])), scv_12
                    )
                ),
            )
        )
    )

<IPython.core.display.Javascript object>

4.Train ensemble modles
===

In [21]:
# stacking classifier
sclf = StackingClassifier(
    classifiers=weakmodles,
    verbose=2,
    meta_classifier=RandomForestClassifier(n_estimators=500),
)
sclf.fit(xtrain, ytrain)
ypred = sclf.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

Fitting 51 classifiers...
Fitting classifier1: pipeline (1/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 4], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
  

Fitting classifier9: pipeline (9/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=

Fitting classifier17: pipeline (17/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 6, 8, 9, 10], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
             

Fitting classifier25: pipeline (25/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 7, 10], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
                   

Fitting classifier33: pipeline (33/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 4, 7], drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)
Fitting classifier34: pipeline (34/51)
Pipeline(memory=None,
         steps=[('columnselec

Fitting classifier42: pipeline (42/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 6, 7, 8, 9, 10], drop_axis=False)),
                ('svc',
                 SVC(C=100, cache_size=None, class_weight={}, coef0=0.0,
                     decision_function_shape='ovo', degree=3, gamma=0.1,
                     gpu_id=0, kernel='linear', max_iter=-1, max_mem_size=-1,
                     n_jobs=-1, probability=False, random_state=None,
                     shrinking=False, tol=0.001, verbose=False))],
         verbose=False)
Fitting classifier43: pipeline (43/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 4, 6, 7, 8, 9, 10, 11],
                                drop_axis=False)),
                ('svc',
                 SVC(C=100, cache_size=None, class_weight={}, coef0=0.0,
                     decision_function_shape='ovo', degree=3, gamma=0.1,
                     gpu_id

,Normal,Prediabetes,diabetes
Normal,153,47,0
Prediabetes,62,130,8
diabetes,3,42,155


<IPython.core.display.Javascript object>

In [22]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.70      0.77      0.73       200
           1       0.59      0.65      0.62       200
           2       0.95      0.78      0.85       200

    accuracy                           0.73       600
   macro avg       0.75      0.73      0.74       600
weighted avg       0.75      0.73      0.74       600



<IPython.core.display.Javascript object>

In [23]:
scores = model_selection.cross_val_score(
    sclf, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Fitting 51 classifiers...
Fitting classifier1: pipeline (1/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 4], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
  

Fitting classifier9: pipeline (9/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=

Fitting classifier17: pipeline (17/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 6, 8, 9, 10], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
             

Fitting classifier25: pipeline (25/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 7, 10], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
                   

Fitting classifier33: pipeline (33/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 4, 7], drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)
Fitting classifier34: pipeline (34/51)
Pipeline(memory=None,
         steps=[('columnselec

...

Fitting classifier16: pipeline (16/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 5, 10, 11], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
            

Fitting classifier24: pipeline (24/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 5, 8, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oo

Fitting classifier32: pipeline (32/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 10], drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)
Fitting classifier33: pipeline (33/51)
Pipeline(memory=None,
         steps=[('columnsele

Fitting classifier41: pipeline (41/51)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 6, 7, 8, 9, 10], drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)
Fitting classifier42: pipeline (42/51)
Pipeline(memory=None,
         steps=[('c

<IPython.core.display.Javascript object>

In [36]:
# stacking classifier
sclf = StackingClassifier(
    classifiers=weakmodles[:25],
    verbose=2,
    meta_classifier=RandomForestClassifier(n_estimators=500),
)
sclf.fit(xtrain, ytrain)
ypred = sclf.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

Fitting 25 classifiers...
Fitting classifier1: pipeline (1/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 5, 10], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state

Fitting classifier9: pipeline (9/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 4, 5, 6, 7, 8, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_scor

Fitting classifier17: pipeline (17/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_scor

Fitting classifier25: pipeline (25/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 6, 8, 10, 11], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
         

,Normal,Prediabetes,diabetes
Normal,147,53,0
Prediabetes,59,127,14
diabetes,4,37,159


<IPython.core.display.Javascript object>

In [37]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.70      0.73      0.72       200
           1       0.59      0.64      0.61       200
           2       0.92      0.80      0.85       200

    accuracy                           0.72       600
   macro avg       0.73      0.72      0.73       600
weighted avg       0.73      0.72      0.73       600



<IPython.core.display.Javascript object>

In [38]:
scores = model_selection.cross_val_score(
    sclf, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Fitting 25 classifiers...
Fitting classifier1: pipeline (1/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 5, 10], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state

Fitting classifier9: pipeline (9/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 4, 5, 6, 7, 8, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_scor

Fitting classifier17: pipeline (17/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_scor

Fitting classifier25: pipeline (25/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 6, 8, 10, 11], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
         

Fitting classifier8: pipeline (8/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 5, 6, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_

...

Fitting classifier25: pipeline (25/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 6, 8, 10, 11], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
         

Fitting classifier8: pipeline (8/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 2, 3, 4, 5, 6, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_

Fitting classifier16: pipeline (16/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 4, 5, 8, 9, 10, 11],
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_s

Fitting classifier24: pipeline (24/25)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=[0, 4], drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=12, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=10,
                                        min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=42,
                          

<IPython.core.display.Javascript object>